In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


examples = [
{
"movie": "매드맥스",
"answer": """
🔥🚗💨
""",
},
{
"movie": "서울의 봄",
"answer": """
🚨🪖🔥
""",
},
]

llm  = ChatOpenAI(model_name="gpt-4",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationBufferMemory(
    return_messages=True,
)

example_prompt = ChatPromptTemplate.from_messages([
        ("human","영화 {movie}을 이모지 3개로 표현해줘"),
        ("ai", "{answer}")
    ]
)

few_shot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 전문가로 줄거리를 이모지로 묘사하는 것으로 유명하지."),
        MessagesPlaceholder(variable_name="history"),
        few_shot_example_prompt,
        ("human", "영화 {movie}을 이모지 3개로 묘사해줘"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]
movie_chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie):
    result = movie_chain.invoke({
        "movie": movie,
    })
    memory.save_context({"input": movie},{"output": result.content}) 
    return result


In [51]:
invoke_chain("Dune")

AIMessageChunk(content='🏜️🐛🚀')

In [52]:
invoke_chain("oldboy")


AIMessageChunk(content='🔨🍣🔄')

In [53]:
invoke_chain("처음에 물어본 영화 제목은?")

AIMessageChunk(content='🏜️🐛🚀')